<a href="https://colab.research.google.com/github/Santosh-Gupta/NaturalLanguageRecommendations/blob/srihari-dev/notebooks/model_debug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers --quiet
%tensorflow_version 2.x

     |████████████████████████████████| 450kB 3.3MB/s 
     |████████████████████████████████| 860kB 50.0MB/s 
     |████████████████████████████████| 1.0MB 39.6MB/s 
TensorFlow 2.x selected.


In [2]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Lambda, Dense, Activation, Concatenate
from transformers import TFBertModel
print('TensorFlow:', tf.__version__)

TensorFlow: 2.1.0-rc1


In [3]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  ['10.54.250.202:8470']
INFO:tensorflow:Initializing the TPU system: 10.54.250.202:8470


INFO:tensorflow:Initializing the TPU system: 10.54.250.202:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [0]:
def get_random_title():
    return tf.random.uniform(shape=[512], maxval=200, dtype=tf.int32)

def get_random_citation():
    return tf.random.uniform(shape=[768], minval=-1, maxval=1, dtype=tf.float32)


def generate_positive_sample():
    title = get_random_title()
    posCitations = tf.stack([get_random_citation() for _ in range(4)], axis=0)
    return title, posCitations

def generate_negative_sample():
    title = get_random_title()
    negCitations = tf.stack([get_random_citation() for _ in range(16)], axis=0)
    return title, negCitations

def merge_datasets(pos, neg):
    title = pos[0]
    posCite = pos[1]
    negCite = neg[1]
    
    posLabels = tf.ones(shape=[4])
    negLabels = tf.zeros(shape=[16])
    labels = tf.concat([posLabels, negLabels], axis=0)
    
    return (title, posCite, negCite), labels

In [0]:
def create_model():
    textInputs = tf.keras.Input(shape=(512,), dtype=tf.int32)
    posCite = tf.keras.Input(shape=(4, 768))
    negCite = tf.keras.Input(shape=(16, 768))
    
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')
    
    textOut = bert_model(textInputs)
    textOutMean = tf.reduce_mean(textOut[0], axis=1)
    textSim = Dense(units=768, activation='tanh', name='DenseText')(textOutMean)
    textSim = tf.tile(tf.expand_dims(textSim, axis=1), multiples=[1, 20, 1])

    Cite = tf.concat([posCite, negCite], axis=1)
    CiteSim = Dense(units=768, activation='tanh', name='DenseCite')(Cite)

    logits = tf.reduce_sum(tf.multiply(textSim, CiteSim), axis=2)
    return tf.keras.Model(inputs=[textInputs, posCite, negCite], outputs=[logits])

In [0]:
batch_size = 3
with strategy.scope():
  model = create_model()
  model.compile(loss=tf.losses.BinaryCrossentropy(from_logits=True),
                optimizer=tf.optimizers.Adam())

In [8]:
with strategy.scope():
  posDataset = tf.data.Dataset.range(5000)
  posDataset = posDataset.shuffle(512)
  posDataset = posDataset.map(lambda _ : generate_positive_sample())
  print(tf.data.experimental.get_structure(posDataset))

  negDataset = tf.data.Dataset.range(5000)
  negDataset = negDataset.shuffle(512)
  negDataset = negDataset.map(lambda _ : generate_negative_sample())
  print(tf.data.experimental.get_structure(negDataset))


  dataset = tf.data.Dataset.zip((posDataset, negDataset))
  dataset = dataset.map(merge_datasets)
  dataset = dataset.batch(batch_size, drop_remainder=True)
  print(tf.data.experimental.get_structure(dataset))

(TensorSpec(shape=(512,), dtype=tf.int32, name=None), TensorSpec(shape=(4, 768), dtype=tf.float32, name=None))
(TensorSpec(shape=(512,), dtype=tf.int32, name=None), TensorSpec(shape=(16, 768), dtype=tf.float32, name=None))
((TensorSpec(shape=(3, 512), dtype=tf.int32, name=None), TensorSpec(shape=(3, 4, 768), dtype=tf.float32, name=None), TensorSpec(shape=(3, 16, 768), dtype=tf.float32, name=None)), TensorSpec(shape=(3, 20), dtype=tf.float32, name=None))


In [10]:
model.summary()
model.fit(dataset, epochs=5, steps_per_epoch=100)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 512, 768), ( 109482240   input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_Mean (TensorFlowOpL [(None, 768)]        0           tf_bert_model[0][0]              
__________________________________________________________________________________________________
DenseText (Dense)               (None, 768)          590592      tf_op_layer_Mean[0][0]           
______________________________________________________________________________________________

100/100 [==============================] - 31s 313ms/step - loss: 0.2389
Epoch 2/5
100/100 [==============================] - 13s 135ms/step - loss: 0.2352
Epoch 3/5
100/100 [==============================] - 14s 136ms/step - loss: 0.2321
Epoch 4/5
100/100 [==============================] - 14s 135ms/step - loss: 0.2256
Epoch 5/5
100/100 [==============================] - 13s 135ms/step - loss: 0.2233
